In [1]:
import pyvista as pv


# Packing CAT Cell demo notebook
The main functions of this demo are located in [chordal_axis_transform.py](../irregular_object_packing/packing/chordal_axis_transform.py)


### Contents
1. Simple CAT Cell demo
2. Larger Demo with cylindrical container and multiple blood cell meshes.

### Pre-requisites
Please install the required packages using the following command:

```bash
pip install -r requirements.txt
```




## Basic CAT Cell demo
The cell below shows a simple demo of computing cat faces. The container is a cube and the object os a cuboid rotated 45 degrees over the z axis. The red faces are the CAT faces that belong to the cube in the back. The yellow arrows are the normal vectors, which point inwards, of each face. the yellow faces are the cat faces that belong to the other objects.

In [9]:
from nis import cat
from irregular_object_packing.cat.chordal_axis_transform import compute_cdt, compute_cat_faces

container = pv.Cube(center=(0, 0, 0), x_length=4, y_length=4, z_length=2).triangulate().clean()

cube1 = pv.Cube(center=(1, 1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube2 = pv.Cube(center=(1, -1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube3 = pv.Cube(center=(-1, 1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube4 = pv.Cube(center=(-1, -1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube5 = pv.Cube(center=(0, 0, 0), x_length=0.9, y_length=0.9, z_length=0.9).triangulate().clean()

meshes = [cube1, cube2, cube3, cube4, cube5]
tetmesh = compute_cdt(meshes + [container])



In [21]:

from irregular_object_packing.mesh.utils import convert_faces_to_polydata_input


n_points_per_obj = [len(mesh.points) for mesh in meshes] + [len(container.points)]
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])



plotter = pv.Plotter()
# plotter.add_mesh_clip_plane(tetmesh, color="grey", show_edges=True, opacity=0.9, crinkle=True)
for mesh in meshes:
    plotter.add_mesh(mesh, color="red", show_edges=True, opacity=1,)

# cat_mesh = pv.PolyData()
# for cat_cell in cat_cells[:-1]:
#     cat_mesh.merge(pv.PolyData(*convert_faces_to_polydata_input(cat_cell)), merge_points=True, inplace=True)
# plotter.add_mesh_clip_plane(cat_mesh, color="yellow", show_edges=True, opacity=0.6, crinkle=True)

# add single cat cell
cell_id = 3
cat_cell = cat_cells[cell_id]
plotter.add_mesh(pv.PolyData(*convert_faces_to_polydata_input(cat_cell)), color="blue", show_edges=True, opacity=0.9,)
plotter.show()

Widget(value="<iframe src='http://localhost:58607/index.html?ui=P_0x17bed7a30_16&reconnect=auto' style='width:…

## Demo computing CAT cells for Blood Cells in a cylindrical container
The cell below shows a demo of computing cat faces for a cylindrical container and multiple blood cell meshes. The yellow faces are the CAT faces and for each red blood cell there is a corresponding CAT cell. 



In [4]:
mesh_volume = 0.1
container_volume = 10
coverage_rate = 0.3

DATA_FOLDER = "../data/mesh/"
loaded_mesh = pv.read(DATA_FOLDER + "RBC_normal.stl")

print_mesh_info(loaded_mesh, "loaded mesh")
# trimesh.Scene([loaded_mesh]).show()

Mesh info loaded mesh: PolyData (0x2863098a0)
  N Cells:    1280
  N Points:   642
  N Strips:   0
  X Bounds:   3.509e+01, 4.291e+01
  Y Bounds:   3.796e+01, 4.004e+01
  Z Bounds:   3.509e+01, 4.291e+01
  N Arrays:   0
, 
volume: 71.04896301964186, 
bounding box: (35.0890007019043, 42.9109992980957, 37.96297836303711, 40.03702163696289, 35.0890007019043, 42.9109992980957) 
center of mass: [39. 39. 39.]



In [5]:
# Scale the mesh to the desired volume
from irregular_object_packing.mesh.transform import scale_and_center_mesh


original_mesh = scale_and_center_mesh(loaded_mesh, mesh_volume)
print_mesh_info(original_mesh, "scaled mesh")

Mesh info scaled mesh: PolyData (0x28630a500)
  N Cells:    1280
  N Points:   642
  N Strips:   0
  X Bounds:   -4.383e-01, 4.383e-01
  Y Bounds:   -1.162e-01, 1.162e-01
  Z Bounds:   -4.383e-01, 4.383e-01
  N Arrays:   0
, 
volume: 0.10000000835585507, 
bounding box: (-0.4382968842983246, 0.43829718232154846, -0.11621677875518799, 0.11621654033660889, -0.438296914100647, 0.4382971525192261) 
center of mass: [ 0.  0. -0.]



In [6]:
container = pv.Sphere(radius=1, center=[0,0,0])
# container = trimesh.primitives.Cylinder(radius=1, height=1)
print_mesh_info(container, "original container")
# container.apply_transform()

container = scale_to_volume(container, container_volume)
print_mesh_info(container, "scaled container")

Mesh info original container: PolyData (0x286308820)
  N Cells:    1680
  N Points:   842
  N Strips:   0
  X Bounds:   -9.985e-01, 9.985e-01
  Y Bounds:   -9.931e-01, 9.931e-01
  Z Bounds:   -1.000e+00, 1.000e+00
  N Arrays:   1
, 
volume: 4.146045958199935, 
bounding box: (-0.9985334277153015, 0.9985334277153015, -0.9930633306503296, 0.9930633306503296, -1.0, 1.0) 
center of mass: [0. 0. 0.]

Mesh info scaled container: PolyData (0x28630ace0)
  N Cells:    1680
  N Points:   842
  N Strips:   0
  X Bounds:   -1.339e+00, 1.339e+00
  Y Bounds:   -1.332e+00, 1.332e+00
  Z Bounds:   -1.341e+00, 1.341e+00
  N Arrays:   1
, 
volume: 9.999999890784448, 
bounding box: (-1.3391151428222656, 1.3391151428222656, -1.3317792415618896, 1.3317792415618896, -1.3410818576812744, 1.3410818576812744) 
center of mass: [0. 0. 0.]



In [7]:
# Initial placement of the objects
objects_coords, _ = init_coordinates(container, original_mesh, coverage_rate=coverage_rate)

scene = create_packed_scene(container, objects_coords, original_mesh, rotate=True)
scene.show()

Widget(value="<iframe src='http://localhost:51881/index.html?ui=P_0x14ee5b3d0_2&reconnect=auto' style='width: …

In [8]:
# get vertices of the object meshes and the container as preparation for CAT computation

# we resample to simplify and get a more uniform distribution of points
from irregular_object_packing.mesh.sampling import resample_pyvista_mesh, resample_pyvista_mesh_kmeans


mesh = resample_pyvista_mesh(original_mesh, 200)
down_scale = 0.1 # initial downscale 
obj_points = []
tf_matrices = []
for i in range(len(objects_coords)):
    # get the object
    object = mesh.copy()
    # random rotation
    M_rot = trimesh.transformations.random_rotation_matrix()
    M_f = trimesh.transformations.scale_matrix(down_scale**(1/3))
    M_t = trimesh.transformations.translation_matrix(objects_coords[i] - np.array([0, 0, 0]))

    M = M_t @ M_f @ M_rot
    # first scale, then rotate, then translate
    points = object.transform(M).points
    # print(points.shape)

    tf_matrices.append(M)
    obj_points.append(points)

container_points = resample_pyvista_mesh_kmeans(container, 200)

In [9]:
# compute the cat cells for each object (takes approx. 1-2 min )
data = compute_cat_cells(obj_points, container_points.points, objects_coords)

In [10]:
# This next part of code is for the visualization of the CAT cells.
# it processes the output of compute_cat_cells to create a PyVista mesh for each object
# and it applies the corresponding transformations on the object meshes.

# Volumetric downscale before optimizing the packing
down_scale = 0.1

object_meshes = []
cat_meshes = []

lim = len(data.cat_faces.keys()) 
for k, v in tqdm(data.cat_faces.items()):
    if k >= lim - 1:
        break
    cat_points, poly_faces = face_coord_to_points_and_faces(data, k)
    polydata = pv.PolyData(cat_points, poly_faces)
    cat_meshes.append(polydata)

    object_mesh = original_mesh.copy()
    object_mesh.transform(tf_matrices[k])
    # object_mesh.apply_scale(down_scale ** (1 / 3))
    # object_mesh.vertices = trimesh.transformations.transform_points(
    #     object_mesh.vertices, translation_matrix(np.array([0, 0, 0]), objects_coords[k])
    # )
    # object_mesh.transform(np.eye(4), object_coords[k])
    object_meshes.append(object_mesh)

 97%|█████████▋| 30/31 [00:00<00:00, 233.46it/s]


In [11]:
create_plot(objects_coords, object_meshes, cat_meshes, container)

Widget(value="<iframe src='http://localhost:51881/index.html?ui=P_0x2986d9750_3&reconnect=auto' style='width: …